<a href="https://colab.research.google.com/github/SURESHBEEKHANI/Advanced-LLM-Fine-Tuning/blob/main/Finetune_Llama_3_2_Vision_OCR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
# The `%%capture` magic in Jupyter/Colab captures output, suppressing it from being displayed.

# Install the `unsloth` package from PyPI
!pip install unsloth

# Uninstall `unsloth` to ensure a clean installation, then install the latest version from GitHub
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git
